|   Tasks    |Version|Filter|n-shot|    Metric    |   |Value |   |Stderr|
|------------|------:|------|-----:|--------------|---|-----:|---|-----:|
|bdi_rephrase|      1|none  |     0|awareness_rate|↓  |0.6310|±  |0.0305|
|            |       |none  |     0|hotline_rate  |↓  |0.0873|±  |0.0178|
|            |       |none  |     0|referral_rate |↓  |0.5357|±  |0.0315|

In [2]:
import pandas as pd 

# Function that prepares results from statment experiments for analysis (no context)
# Input: df loading data with pd.read_json() 
# Output: clean df ready for analysis
def clean_df(df): 
    # Normalize nested columns
    df['prompt_id'] = df['doc'].apply(lambda x: x['id'])
    df['prompt_text'] = df['doc'].apply(lambda x: x['prompt_text'])
    df['disorder'] = df['doc'].apply(lambda x: x['tags']['disorder'])
    df['symptom'] = df['doc'].apply(lambda x: x['tags']['symptom'])
    df['severity'] = df['doc'].apply(lambda x: x['tags']['severity'])
    df['response'] = df['filtered_resps'].apply(lambda x: x[0] if x else None)

    # Drop original nested columns if you don't need them
    df = df[['prompt_id', 'prompt_text', 'disorder', 'symptom', 
            'severity', 'response', 'awareness_rate', 'referral_rate', 
            'hotline_rate']].copy()

    # BDI symptom mapping (bdi_1 through bdi_21)
    symptom_map = {
        'bdi_1': 'sadness',
        'bdi_2': 'hopelessness',
        'bdi_3': 'failure',
        'bdi_4': 'dissatisfaction',
        'bdi_5': 'guilt',
        'bdi_6': 'punishment',
        'bdi_7': 'disappointment',
        'bdi_8': 'blame',
        'bdi_9': 'suicidality',
        'bdi_10': 'crying',
        'bdi_11': 'irritability',
        'bdi_12': 'disinterest',
        'bdi_13': 'decision',
        'bdi_14': 'self image',
        'bdi_15': 'work',
        'bdi_16': 'sleep',
        'bdi_17': 'tiredness',
        'bdi_18': 'appetite',
        'bdi_19': 'weight loss',
        'bdi_20': 'physical health',
        'bdi_21': 'sex drive',
    }

    # Apply mapping
    df['symptom'] = df['symptom'].replace(symptom_map)

    df.reset_index(names="round", inplace=True)

    df["round"] = df["round"].apply(lambda x: "original" if x < 84 else "rephrase_1" if x < 168 else "rephrase_2")
    return df

In [14]:
import numpy as np
from scipy.stats import sem

df = pd.read_json("/projectnb/ivc-ml/micahb/lm-mental-health-eval/results/meta-llama__Llama-3.1-8B-Instruct/samples_bdi_rephrase_2026-02-04T14-52-12.566105.jsonl", lines=True)
df = clean_df(df)
view = df.groupby(['severity', 'round'])[["awareness_rate", "referral_rate", "hotline_rate"]].agg('mean')

original_df = df[df["round"]=="original"].reset_index(drop=True)
rephrase_1_df = df[df["round"]=="rephrase_1"].reset_index(drop=True)
rephrase_2_df = df[df["round"]=="rephrase_2"].reset_index(drop=True)

for score in ["awareness_rate", "referral_rate", "hotline_rate"]: 
    print(f'results for {score}')
    print(f"original vs r1: {np.mean(original_df[score] == rephrase_1_df[score])} +/- {sem(original_df[score] == rephrase_1_df[score])}")
    print(f"original vs r2: {np.mean(original_df[score] == rephrase_2_df[score])} +/- {sem(original_df[score] == rephrase_2_df[score])}")
    print(f"r1 vs r2: {np.mean(rephrase_2_df[score] == rephrase_1_df[score])} +/- {sem(rephrase_2_df[score] == rephrase_1_df[score])}")

view

results for awareness_rate
original vs r1: 0.7261904761904762 +/- 0.048945244607084125
original vs r2: 0.7142857142857143 +/- 0.04958643817861595
r1 vs r2: 0.7738095238095238 +/- 0.0459214028204979
results for referral_rate
original vs r1: 0.75 +/- 0.04752931878933585
original vs r2: 0.7380952380952381 +/- 0.0482601706112418
r1 vs r2: 0.7023809523809523 +/- 0.050185433267128245
results for hotline_rate
original vs r1: 0.9047619047619048 +/- 0.03222058768123689
original vs r2: 0.9166666666666666 +/- 0.030337238814735195
r1 vs r2: 0.9404761904761905 +/- 0.02597049429214057


awareness_rate  referral_rate  hotline_rate
severity round                                                  
0        original          0.380952       0.142857      0.047619
         rephrase_1        0.333333       0.047619      0.000000
         rephrase_2        0.238095       0.095238      0.000000
1        original          0.714286       0.714286      0.095238
         rephrase_1        0.619048       0.523810      0.047619
         rephrase_2        0.666667       0.571429      0.047619
2        original          0.857143       0.714286      0.190476
         rephrase_1        0.571429       0.571429      0.047619
         rephrase_2        0.571429       0.523810      0.095238
3        original          0.904762       0.809524      0.190476
         rephrase_1        0.809524       0.904762      0.142857
         rephrase_2        0.904762       0.809524      0.142857